In [1]:
import os


# Environment setup
MAXIM_API_KEY = os.getenv("MAXIM_API_KEY")
LOG_REPOSITORY_ID = os.getenv("MAXIM_LOG_REPO_ID")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")
MODEL_NAME = "meta-llama/Meta-Llama-3.2-8B"

if not all([MAXIM_API_KEY, LOG_REPOSITORY_ID, TOGETHER_API_KEY]):
    raise EnvironmentError("API keys and Log Repository ID are not set in the environment variables.")

In [2]:
from uuid import uuid4
from maxim import Maxim

logger = Maxim().logger({"id": LOG_REPOSITORY_ID})

In [3]:
session = logger.session({
    "id": str(uuid4()),
})
trace = session.trace({
    "id": str(uuid4()),
})

In [4]:
from together import Together
from maxim.logger import  GenerationError
import time

# Together AI client setup
client = Together(api_key=TOGETHER_API_KEY)

def get_together_response(messages):
    generation_id = str(uuid4())
    generation = trace.generation({
        "id": generation_id,
        "name": "generation",
        "provider": "together",
        "model": MODEL_NAME,
        "messages": messages
    })
    
    try:
        # Get response from Together AI
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages
        )
        
        response_text = response.choices[0].message.content
        
        # Log the generation result
        generation.result({
            "id": generation_id,
            "object": "chat.completion",
            "created": int(time()),
            "model": MODEL_NAME,
            "choices": [
                {
                    "index": 0,
                    "text": response_text,
                    "logprobs": None,
                    "finish_reason": "stop",
                }
            ],
            "usage": {
                "prompt_tokens": getattr(response, 'usage', {}).get('prompt_tokens', 0),
                "completion_tokens": getattr(response, 'usage', {}).get('completion_tokens', 0),
                "total_tokens": getattr(response, 'usage', {}).get('total_tokens', 0),
            }
        })
        
        return response_text
        
    except Exception as e:
        generation.error(GenerationError(str(e)))
        raise
        
    finally:
        generation.end()


In [5]:
messages = [{"role": "user", "content": "What are some fun things to do in New York?"}]
trace.event(str(uuid4()), "test event")
# Get and log response
completion = get_together_response(messages)

# Print result
print("\nTogether AI Response:")
print(completion)

InvalidRequestError: Error code: 404 - {"message": "Unable to access model meta-llama/Meta-Llama-3.2-8B. Please visit https://api.together.ai/models to view the list of supported models.", "type_": "invalid_request_error", "code": "model_not_available"}